In [ ]:
import git

In [14]:
repo = git.Repo('')
repo.git.add(update=True)


''

In [15]:
repo.index.commit('Automatic Upload')

<git.Commit "ef53a8a02a5632fae620c7c0d68f906e3a88c8d3">

In [16]:
origin = repo.remote(name='origin')
origin.push()

In [6]:
g.push()

''

In [9]:
g.commit()

GitCommandError: Cmd('git') failed due to: exit code(1)
  cmdline: git commit
  stdout: 'On branch master
Your branch is up-to-date with 'origin/master'.
Changes not staged for commit:
	modified:   config.py
	modified:   run.sh

Untracked files:
	Notes.ipynb
	pygit_logger.log

no changes added to commit'